<a href="https://colab.research.google.com/github/imdeepmind/UdacitySelfDrivingCar/blob/master/Base_Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Getting the into colab
!git clone https://github.com/imdeepmind/UdacitySelfDrivingCar.git
!mkdir data
!mv UdacitySelfDrivingCar/* ./
!rm -r UdacitySelfDrivingCar

Cloning into 'UdacitySelfDrivingCar'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 5233 (delta 18), reused 10 (delta 5), pack-reused 5200
Receiving objects: 100% (5233/5233), 66.03 MiB | 10.83 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [1]:
!pip install neuralpy-torch

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import cv2
import torch

from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.transforms as transforms

from neuralpy.models import Sequential
from neuralpy.layers import Conv2D, Dense, Flatten, MaxPool2D
from neuralpy.regularizers import Dropout2D
from neuralpy.activation_functions import ReLU

In [3]:
# Constants
CSV_FILE = './data/driver_log_cleaned.csv'
BATCH_SIZE=32

In [4]:
# Reading CSV file
data = pd.read_csv(CSV_FILE)
data.head()

,center_camera,left_camera,right_camera,steering,throttle,reverse,speed
0,./data/IMG/center_2020_09_05_16_40_17_113.jpg,./data/IMG/left_2020_09_05_16_40_17_113.jpg,./data/IMG/right_2020_09_05_16_40_17_113.jpg,0.0,0.0,0,0.000077
1,./data/IMG/center_2020_09_05_16_40_17_214.jpg,./data/IMG/left_2020_09_05_16_40_17_214.jpg,./data/IMG/right_2020_09_05_16_40_17_214.jpg,0.0,0.0,0,0.000077
2,./data/IMG/center_2020_09_05_16_40_17_320.jpg,./data/IMG/left_2020_09_05_16_40_17_320.jpg,./data/IMG/right_2020_09_05_16_40_17_320.jpg,0.0,0.0,0,0.000077
3,./data/IMG/center_2020_09_05_16_40_17_425.jpg,./data/IMG/left_2020_09_05_16_40_17_425.jpg,./data/IMG/right_2020_09_05_16_40_17_425.jpg,0.0,0.0,0,0.000077
4,./data/IMG/center_2020_09_05_16_40_17_531.jpg,./data/IMG/left_2020_09_05_16_40_17_531.jpg,./data/IMG/right_2020_09_05_16_40_17_531.jpg,0.0,0.0,0,0.000077


In [5]:
left_image = data["left_camera"].values
center_image = data["center_camera"].values
right_image = data["right_camera"].values

steering = data["steering"].values

In [6]:
# Custom Data Loader class
class DriveDataset(Dataset):
  def __init__(self, left_images, center_images, right_images, steering, transform=None):
    self.__left_images = left_images
    self.__center_images = center_images
    self.__right_images = right_images

    self.__steering = steering

    self.__transform = transform
  
  def __len__(self):
    return len(self.__steering)
  
  def __getitem__(self, index):
    left = cv2.imread(self.__left_images[index], 1)
    center = cv2.imread(self.__center_images[index], 1)
    right = cv2.imread(self.__right_images[index], 1)

    stacked = np.hstack([left, center, right])

    if self.__transform:
      stacked = self.__transform(stacked)

    steering =  torch.tensor(float(self.__steering[index]))

    return stacked, steering

In [7]:
# Making the data loader generator
dataset = DriveDataset(left_image, center_image, right_image, steering, transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(left_image) * .8), int(len(steering) - (len(steering) * .8) + 1) ])

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

In [19]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, input_shape=(3, 160,960)))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=64, kernel_size=3, input_shape=(3, 160,960)))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=128, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=128, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=256, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=256, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Flatten())

model.add(Dense(1024))
model.add(ReLU())

model.add(Dense(1024))
model.add(ReLU())

model.add(Dense(1))

In [20]:
model.build()
model.summary()

The model is running on cpu
Sequential(
  (conv2d_layer_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_2): ReLU()
  (maxpool2d_layer_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_4): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_5): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_6): ReLU()
  (maxpool2d_layer_7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_8): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_10): ReLU()
  (maxpool2d_layer_11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_12): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_14): ReLU()
  (maxpool2d_layer_15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_lay

In [21]:
pytorch_model = model.get_model()
pytorch_model

Sequential(
  (conv2d_layer_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_2): ReLU()
  (maxpool2d_layer_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_4): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_5): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_6): ReLU()
  (maxpool2d_layer_7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_8): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_10): ReLU()
  (maxpool2d_layer_11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_12): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_14): ReLU()
  (maxpool2d_layer_15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_16): Dropout2d(p=0.5, inp